### Tutorial: Basic Aggregations, Group By, Window Functions
Covers
* Aggregations
* Group By
* Window Functions

This isn't part of a project, this simple demonstrations of the various tools in Spark / PostGresQL

In [0]:
%run "./includes/config_file_paths"

In [0]:
race_results_df=spark.read.parquet(f"{presentation_folder_path}/race_results")

In [0]:
demo_df=race_results_df.filter("race_year=2020")

## Aggregations
Sum count, min, max, etc....

Documentation:
* https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.sql.html#functions

and then search for "Aggregate"

In [0]:
from pyspark.sql.functions import sum, when, count, col, countDistinct


In [0]:
demo_df.select(count("*")).show()

+--------+
|count(1)|
+--------+
|     340|
+--------+



##### Number of distinct races in 2020

In [0]:
demo_df.select(countDistinct("race_name")).show()

+-------------------------+
|count(DISTINCT race_name)|
+-------------------------+
|                       17|
+-------------------------+



##### Sum of Points

In [0]:
demo_df.select(sum("points")).show()

+-----------+
|sum(points)|
+-----------+
|     1734.0|
+-----------+



##### How many points Lewis Hamilton scored

In [0]:
demo_df.filter("driver_name = 'Lewis Hamilton'").select(sum("points")).show()

+-----------+
|sum(points)|
+-----------+
|      347.0|
+-----------+



##### Lewis Hamilton had Covid in 2020 so he's missing 1 race 16 instead of 17

In [0]:
demo_df.filter("driver_name = 'Lewis Hamilton'").select(sum("points"), countDistinct("race_name")).show()

+-----------+-------------------------+
|sum(points)|count(DISTINCT race_name)|
+-----------+-------------------------+
|      347.0|                       16|
+-----------+-------------------------+



#### Number of distinct races in 2020

In [0]:
demo_df.filter("driver_name = 'Lewis Hamilton'").select(sum("points"), countDistinct("race_name")) \
    .withColumnRenamed("sum(points)","total_points") \
    .withColumnRenamed("count(DISTINCT race_name)","number_of_races") \
    .show()

+------------+---------------+
|total_points|number_of_races|
+------------+---------------+
|       347.0|             16|
+------------+---------------+



### Group Aggregations

In [0]:
demo_df.filter("driver_name = 'Lewis Hamilton'").select(sum("points"), countDistinct("race_name")) \
    .withColumnRenamed("sum(points)","total_points") \
    .withColumnRenamed("count(DISTINCT race_name)","number_of_races") \
    .show()

+------------+---------------+
|total_points|number_of_races|
+------------+---------------+
|       347.0|             16|
+------------+---------------+



### Group By

You can't do group by after sum() as that changes the datatype to Pandas DataFrame

```
demo_df \
    .groupBy("driver_name") \
    .sum("points") 

Out[21]: DataFrame[driver_name: string, sum(points): double]

```
so you must use .agg()


https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.agg.html#pyspark.sql.DataFrame.agg

In [0]:
demo_df \
    .groupBy("driver_name") \
    .sum("points") 

Out[13]: DataFrame[driver_name: string, sum(points): double]

In [0]:
demo_df \
    .groupBy("driver_name") \
    .agg(sum("points"), countDistinct("race_name")) \
    .withColumnRenamed("sum(points)","total_points") \
    .withColumnRenamed("count(DISTINCT race_name)","number_of_races") \
    .show()

+------------------+------------+----------------+
|       driver_name|total_points|count(race_name)|
+------------------+------------+----------------+
|       Jack Aitken|         0.0|               1|
|      Daniil Kvyat|        32.0|              17|
|   Kevin Magnussen|         1.0|              17|
|      Sergio Pérez|       125.0|              15|
|      Carlos Sainz|       105.0|              17|
|    Kimi Räikkönen|         4.0|              17|
|   Romain Grosjean|         2.0|              15|
|   Charles Leclerc|        98.0|              17|
|   Alexander Albon|       105.0|              17|
|      Lance Stroll|        75.0|              16|
|      Pierre Gasly|        75.0|              17|
|    Lewis Hamilton|       347.0|              16|
|   Nico Hülkenberg|        10.0|               3|
|  Daniel Ricciardo|       119.0|              17|
|   Valtteri Bottas|       223.0|              17|
|Antonio Giovinazzi|         4.0|              17|
|      Lando Norris|        97.

##### Shortcut rename with alias!

In [0]:
demo_df \
    .groupBy("driver_name") \
    .agg(sum("points").alias("total_points"), countDistinct("race_name").alias("number_of_races")) \
    .show()

+------------------+------------+---------------+
|       driver_name|total_points|number_of_races|
+------------------+------------+---------------+
|       Jack Aitken|         0.0|              1|
|      Daniil Kvyat|        32.0|             17|
|   Kevin Magnussen|         1.0|             17|
|      Sergio Pérez|       125.0|             15|
|      Carlos Sainz|       105.0|             17|
|    Kimi Räikkönen|         4.0|             17|
|   Romain Grosjean|         2.0|             15|
|   Charles Leclerc|        98.0|             17|
|   Alexander Albon|       105.0|             17|
|      Lance Stroll|        75.0|             16|
|      Pierre Gasly|        75.0|             17|
|    Lewis Hamilton|       347.0|             16|
|   Nico Hülkenberg|        10.0|              3|
|  Daniel Ricciardo|       119.0|             17|
|   Valtteri Bottas|       223.0|             17|
|Antonio Giovinazzi|         4.0|             17|
|      Lando Norris|        97.0|             17|


### Window Functions

Go to API Reference
* https://spark.apache.org/docs/3.1.1/api/python/reference/index.html?highlight=api%20reference

Click on Window
* https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.sql.html#window

In [0]:
demo_df=race_results_df.filter("race_year in (2019,2020)")

In [0]:
demo_df \
    .groupBy("race_year","driver_name") \
    .agg(sum("points").alias("total_points"), countDistinct("race_name").alias("number_of_races"))

Out[24]: DataFrame[race_year: int, driver_name: string, total_points: double, number_of_races: bigint]

In [0]:
demo_grouped_df = demo_df \
    .groupBy("race_year","driver_name") \
    .agg(sum("points").alias("total_points"), countDistinct("race_name").alias("number_of_races"))

In [0]:
display(demo_grouped_df)


race_year,driver_name,total_points,number_of_races
2020,Daniil Kvyat,32.0,17
2019,Kevin Magnussen,20.0,21
2020,Kevin Magnussen,1.0,17
2020,Antonio Giovinazzi,4.0,17
2020,Nico Hülkenberg,10.0,3
2020,Romain Grosjean,2.0,15
2019,Robert Kubica,1.0,21
2020,Charles Leclerc,98.0,17
2019,Lance Stroll,21.0,21
2020,Esteban Ocon,62.0,17


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

driverRankSpec = Window.partitionBy("race_year").orderBy(desc("total_points"))
demo_grouped_df.withColumn("rank", rank().over(driverRankSpec)).show(25)

+---------+------------------+------------+---------------+----+
|race_year|       driver_name|total_points|number_of_races|rank|
+---------+------------------+------------+---------------+----+
|     2019|    Lewis Hamilton|       413.0|             21|   1|
|     2019|   Valtteri Bottas|       326.0|             21|   2|
|     2019|    Max Verstappen|       278.0|             21|   3|
|     2019|   Charles Leclerc|       264.0|             21|   4|
|     2019|  Sebastian Vettel|       240.0|             21|   5|
|     2019|      Carlos Sainz|        96.0|             21|   6|
|     2019|      Pierre Gasly|        95.0|             21|   7|
|     2019|   Alexander Albon|        92.0|             21|   8|
|     2019|  Daniel Ricciardo|        54.0|             21|   9|
|     2019|      Sergio Pérez|        52.0|             21|  10|
|     2019|      Lando Norris|        49.0|             21|  11|
|     2019|    Kimi Räikkönen|        43.0|             21|  12|
|     2019|   Nico Hülken